In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Import libraries and read the data

In [ ]:
import random
import pandas as pd 
import numpy as np
import math as ma
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

#Read the Data
df=pd.read_csv('../input/titanic/train.csv')
df_test=pd.read_csv('../input/titanic/test.csv')

# Do some analysis on data
 * show the number of null samples in train , test datasets.
 * we will notice here that columns (age,cabin) have a great number of null values so we will    drop it.
 

In [ ]:
print(df.isna().sum())
print(df_test.isna().sum())

# Describe the data and show some statstics.

In [ ]:
df.describe()

# Do some preprocessing on train and test datasets
* drop unimportant columns.
* fill null values in embarked column with random choice.
* divide my data to trainning features and label.
* scale my data for easy trainig and convergence.

In [ ]:
#preprocessing on main df 
df.drop(columns=['Cabin','Age', 'PassengerId','Name','Ticket','Parch','SibSp'],inplace=True)



df["Embarked"].fillna( random.choice(df[df['Embarked'] != np.nan]["Embarked"]), inplace =True)


from sklearn.preprocessing import LabelEncoder
df=df.apply(LabelEncoder().fit_transform)


#divide data to training and label
X=df.iloc[ : ,1:].values
y=df.iloc[:,0]



sc = StandardScaler()
X = sc.fit_transform(X)
print(X)


# Repeat the same steps on testset.

In [ ]:
#preprocessing on test df 

df_test.drop(columns=['Cabin','Age', 'PassengerId','Name','Ticket','Parch','SibSp'],inplace=True)


df_test['Fare'].fillna((df_test['Fare'].mean()), inplace=True)
df_test["Embarked"].fillna( random.choice(df_test[df_test['Embarked'] != np.nan]["Embarked"]), inplace =True)








In [ ]:




from sklearn.preprocessing import LabelEncoder
df_test=df_test.apply(LabelEncoder().fit_transform)

# Feature Scaling
sc = StandardScaler()
df_test = sc.fit_transform(df_test)
print(df_test)


Do some visulization on tra

In [ ]:
import seaborn as sns
sns.pairplot(df)


In [ ]:
boxplot = df.boxplot(column=[ 'Fare']) 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

bagging = BaggingClassifier(AdaBoostClassifier(n_estimators=3),max_samples=0.5, max_features=0.5)
bagging.fit(X_train,y_train)
ad_y_pred = bagging.predict(X_test)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, ad_y_pred)
print(cm)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier()
RF.fit(X_train,y_train)
rf_y_pred=RF.predict(X_test)
cm = confusion_matrix(y_test, rf_y_pred)
print(cm)

In [ ]:
final_test=(rf_y_pred+ad_y_pred)/2
final_test = (final_test > 0.5)
final_test

cm = confusion_matrix(y_test, final_test)
print(cm)

In [ ]:
from sklearn.metrics import precision_score,recall_score
recall=recall_score(y_pred,final_test)
precision=precision_score(y_pred,final_test)
print('recall is '+ str(recall))
print('precision is '+ str(precision))


In [ ]:
df_test

In [ ]:
final=(bagging.predict(df_test)+RF.predict(df_test))/2
final=np.where(final >.5 , final , 0)
final

In [ ]:
test = pd.read_csv(r"../input/titanic/test.csv")
submission = pd.DataFrame({
    "PassengerId" : test["PassengerId"],
    "Survived" : final
})
submission.head(10)

In [ ]:
submission.to_csv('submission.csv', index = False)
